# 01 Data Ingestion and Quality Checks

## Objectives

- Load the raw dataset from the versioned folder
- Perform basic data quality checks and standardization
- Export a cleaned dataset to the processed folder

## Inputs

- data/raw/v1/environmental_trends.csv

## Outputs

- data/processed/v1/environmental_trends_clean.csv

## Additional Comments

- Keep all changes reproducible and logged in the notebook

---

# Change working directory

We store notebooks in a subfolder, so we set the project root as the working directory.

In [ ]:
import os
current_dir = os.getcwd()
current_dir

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("Current working directory set to project root")

In [ ]:
os.getcwd()

# Load raw data

In [ ]:
import pandas as pd
raw_path = "data/raw/v1/environmental_trends.csv"
df = pd.read_csv(raw_path)
df.head()

# Quality checks

In [ ]:
df.info()

In [ ]:
df.isna().sum().sort_values(ascending=False).head(10)

In [ ]:
df.duplicated().sum()

# Save cleaned data

In [ ]:
clean_path = "data/processed/v1/environmental_trends_clean.csv"
df.to_csv(clean_path, index=False)
clean_path